<a href="https://colab.research.google.com/github/PratiteeMalakar/hello-world/blob/main/multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install langchain openai google-api-python-client streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.

In [3]:
pip install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:00


In [4]:
%pip install -qU "langchain-community>=0.2.11" tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.3 MB/s eta 0:00:00


In [5]:
from tavily import TavilyClient
from google.colab import userdata
import os

os.environ["TAVILY_API_KEY"]=userdata.get("TAVILY")

api_key = os.getenv("TAVILY_API_KEY")

# Step 1. Instantiating your TavilyClient
tavily_client = TavilyClient(api_key=api_key)

# Step 2. Executing a simple search query
# response = tavily_client.search("Who is Leo Messi?")

# # Step 3. That's it! You've done a Tavily Search!
# print(response)

# Step 2. Executing a context search query
# context = tavily_client.get_search_context(query="What happened during the Burning Man floods?")

# # Step 3. That's it! You now have a context string that you can feed directly into your RAG Application
# print(context)

In [6]:
pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 3.0 MB/s eta 0:00:00


In [7]:


from langchain.adapters.openai import convert_openai_messages
from langchain_community.chat_models import ChatOpenAI
from tavily import TavilyClient
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain_core.messages import ToolMessage
from langchain.schema.messages import BaseMessage
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode
from typing import Sequence, TypedDict, Literal
from typing_extensions import Annotated
import functools

In [10]:
pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.7 MB/s eta 0:00:00


In [11]:
# Set up OpenAI LLM
from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"]=userdata.get("OPENAI")


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model='gpt-4', openai_api_key=OPENAI_API_KEY, temperature=0.0)


In [8]:
def create_agent(llm, tools, system_message: str):
    """Create an agent"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert researcher, collaborating with other assistant researchers, all skilled at researching private companies and producing informative, descriptive, and factual analysis."
                " If you are unable to fully answer, that's okay, other assistants with different tools will"
                " help with where you left off."
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    return llm.bind_tools(tools, prompt=prompt.partial(system_message=system_message))


In [12]:
# Tools (in this case, Tavily search)
def tavily_tool(query):
    return tavily_client.search(query, search_depth="advanced")["results"]

In [13]:
def agent_node(state, agent, name):
    if name == "research_supervisor":
        state["supervisor_invocations"] += 1
        if state["supervisor_invocations"] > 5:
            state["finalizing"] = True

    if state['finalizing']:
        state["messages"].append(HumanMessage(content="Conclude research and compile all the information provided by other assistants and organize it as a company research report. Prefix the answer with 'FINAL ANSWER'."))

    result = agent.invoke(state)
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        "sender": name,
        "supervisor_invocations": state["supervisor_invocations"],
        "finalizing": state["finalizing"],
    }


In [14]:
def router(state) -> Literal["call_tool", "__end__", "continue"]:
    messages = state["messages"]
    last_message = messages[-1]
    if state["finalizing"]:
        return "__end__"
    if last_message.tool_calls:
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        return "__end__"
    return "continue"


In [15]:
research_supervisor_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You are the the most senior and experienced private equity company research manager. You have a team of assistants - 'company_overview_research_agent', 'financial_research_agent', 'business_model_agent', 'key_products_or_services_researcher_agent'. For the company input, delegate each task to different assistants. Your task is to compile all the information provided by other assistants and organize it as a company research report. The final answer should include all the necessary information in well formatted manner."
)
research_supervisor_node = functools.partial(agent_node, agent=research_supervisor_agent, name="research_supervisor")

company_overview_research_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You are a curious and passionate private equity researcher. You should provide accurate information about the company name input such that it covers the 'Company Overview' part of a research.",
)
research_node = functools.partial(agent_node, agent=company_overview_research_agent, name="company_overview_researcher")

financial_research_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You are a skilled financial analyst skilled at researching and scraping financial information about private companies through publicly available data. For the company input, provide any relevant financial data about the company. If using tavily_tool, suffix search text with 'company financials'.",
)
financial_research_node = functools.partial(agent_node, agent=financial_research_agent, name="financial_researcher")

business_model_agent = create_agent(
    llm,
    [tavily_tool],
    system_message="You are an expert consultant skilled at understanding business models of different companies. For the company input, provide accurate business model based on analysis."
)
business_model_research_node = functools.partial(agent_node, agent=business_model_agent, name="business_model_researcher")


AttributeError: 'function' object has no attribute 'name'